In [1]:
import cv2 as cv
import numpy as np

In [2]:
net = cv.dnn.readNet("./config/yolov3.weights", "./config/yolov3.cfg")
classes = []
with open("./data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [10]:
# Loading image
cap = cv.VideoCapture('./Videos/Video1.mp4')
print(cap.isOpened())
while True :
    _, frame = cap.read()
    print(frame)
    height, width, channels = frame.shape
    
    # Detecting objects
    blob = cv.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = (255, 0, 0)
            cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv.putText(frame, label, (x, y + 30), font, 3, color, 3)
            faceCascade = cv.CascadeClassifier('./config/haarcascade_frontalface_default.xml')
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv.CASCADE_SCALE_IMAGE)
            for (x, y ,w, h) in faces:
                cv.rectangle(frame, (x, y), (x + w, y + h),(0, 255, 0), 2)
                cv.putText(frame, "Face", (x, y + 40), font, 3, (0, 0, 255), 2)
    cv.imshow("Image", frame)
    key = cv.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv.destroyAllWindows()

True
[[[ 29  38   9]
  [ 25  34   5]
  [ 22  44   0]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[ 31  40  11]
  [ 29  38   9]
  [ 23  45   1]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[ 28  41  15]
  [ 31  44  18]
  [ 27  47  10]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[ 39  34  40]
  [ 41  36  42]
  [ 43  39  42]
  ...
  [ 22   0   8]
  [ 23   0   9]
  [ 23   0   9]]

 [[ 38  33  39]
  [ 40  35  41]
  [ 42  38  41]
  ...
  [ 26   1  12]
  [ 27   2  13]
  [ 27   2  13]]

 [[ 36  31  37]
  [ 39  34  40]
  [ 42  38  41]
  ...
  [ 35  10  21]
  [ 36  11  22]
  [ 36  11  22]]]
[[[ 22  36  16]
  [ 22  36  16]
  [ 14  34  15]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[ 24  38  18]
  [ 24  38  18]
  [ 18  38  19]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[ 26  42  17]
  [ 26  42  17]
  [ 23  40  22]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[ 40  37  36]
  [ 40  37  36]
  [ 40  37  36]

In [5]:
cap.release()